更新于北京时间2025年1月9日02:00

说明：

1.此程序下使用的是TensorRT后端引擎。如需使用CUDA后端，请进入此链接：
https://colab.research.google.com/drive/1xs3oYijFz6Pkdzn9VBlOi4gC-Qd8nXC_?usp=sharing

2.使用之前先选择使用T4 GPU。会有额外约两分钟的配置环境时间，请耐心等待。已为28b与HumanSL提供plan cache，连接即用。线程已调到最佳，无需修改。

3.连接的命令行为：

ikatago.exe -p all -u 用户名 --password 密码

注意这里用的platform不是colab，与以前的不同。

4.配置环境后包中有28b 18b 40b 30b 20b权，可以在命令行中使用--kata-weight 18b等等指定。即：

ikatago.exe -p all -u 用户名 --password --kata-weight 18b

5.需要使用自己的权重就在下面一个单元格最后一行的wget后面加上权重下载链接，但权重必须以.bin.gz结尾

6.支持humansl，但只能用28b权重。8D以下命令行为：（以8段水平为例）

ikatago.exe -p all -u 用户名 --password 密码 --kata-name humansl8d

启动引擎后可在gtp控制区输入kata-set-param humanSLProfile rank_8d，将8d改为需要的等级即可，例如20k、5k、1d、5d等

9d的命令行这样写：

ikatago.exe -p all -u 用户名 --password 密码 --kata-name humansl9d

In [ ]:
# 配置环境（务必启动这个单元格！等运行完毕再点下面的单元格）
downloadURL='https://github.com/FengmingGo/katago_for_colaboratory/releases/' # 后面除了katago权重、libzip之外其他所有文件都在此目录下，Ctrl+单击可进入。
# 装libssl
%cd ~
!wget {downloadURL}download/20241228/libssl1.1_1.1.1f-1ubuntu2.17_amd64.deb
!dpkg -i libssl1.1_1.1.1f-1ubuntu2.17_amd64.deb && rm libssl1.1_1.1.1f-1ubuntu2.17_amd64.deb
# 装libzip
!wget https://libzip.org/download/libzip-1.11.2.tar.gz
!tar -zxvf libzip-1.11.2.tar.gz
%cd libzip-1.11.2
!cmake .
!make && make install
!ldconfig /usr/local/lib && ldconfig /usr/local/lib64
%cd ..
!rm -rf libzip-1.11.2 && rm libzip-1.11.2.tar.gz
# 装TensorRT
!apt-get install libnvinfer10=10.7.0.23-1+cuda12.6
# 下载ikatago相关文件、plan cache与katago权重
!wget {downloadURL}download/20250107/tensorrt.zip
!unzip tensorrt && rm tensorrt.zip
!mkdir -p /root/.katago/trtcache
!wget {downloadURL}download/20241231/trt-100700_gpu-131365bf_net-kata1-b28c512nbt-s8032072448-d4548958859_5_exact19x19_batch12_fp16 -O ~/.katago/trtcache/trt-100700_gpu-131365bf_net-kata1-b28c512nbt-s8032072448-d4548958859_5_exact19x19_batch12_fp16
!wget {downloadURL}download/20250107/trt-100700_gpu-131365bf_net-b18c384nbt-humanv0_5_exact19x19_batch8_fp16 -O ~/.katago/trtcache/trt-100700_gpu-131365bf_net-b18c384nbt-humanv0_5_exact19x19_batch8_fp16
!wget {downloadURL}download/20250107/trt-100700_gpu-131365bf_net-kata1-b28c512nbt-s8032072448-d4548958859_5_exact19x19_batch8_fp16 -O ~/.katago/trtcache/trt-100700_gpu-131365bf_net-kata1-b28c512nbt-s8032072448-d4548958859_5_exact19x19_batch8_fp16
!chmod -R 777 ./work/
!wget https://github.com/lightvector/KataGo/releases/download/v1.15.0/b18c384nbt-humanv0.bin.gz -O ./work/data/humansl.bin.gz
!wget https://media.katagotraining.org/uploaded/networks/models/kata1/kata1-b28c512nbt-s8032072448-d4548958859.bin.gz -O ./work/data/28b.bin.gz
!wget https://media.katagotraining.org/uploaded/networks/models/kata1/kata1-b18c384nbt-s9996604416-d4316597426.bin.gz -O ./work/data/18b.bin.gz
!wget https://media.katagotraining.org/uploaded/networks/models/kata1/kata1-b40c256-s11840935168-d2898845681.bin.gz -O ./work/data/40b.bin.gz
!wget https://github.com/lightvector/KataGo/releases/download/v1.4.5/g170-b30c320x2-s4824661760-d1229536699.bin.gz -O ./work/data/30b.bin.gz
!wget https://media.katagotraining.org/uploaded/networks/models/kata1/kata1-b20c256x2-s5303129600-d1228401921.bin.gz -O ./work/data/20b.bin.gz
!wget
print('配置环境完毕，可启动ikatago server')

In [ ]:
# 启动ikatago server
username="ikatago" # 将双引号中的ikatago修改为自己需要的ikatago用户名
password="kinfkong" # 将双引号中的kinfkong修改为自己需要的ikatago连接密码
%cd ~/work
!./run.sh {username} {password}

In [ ]:
!lsb_release -a # 获取系统版本等信息
!nvidia-smi # 获取GPU及其利用率等信息
!nvcc -V # 查询CUDA版本等信息

In [ ]:
# benchmark，前提是在运行完第一个单元格后进行
model='28b' # 默认可填18b、28b、40b、30b、20b
visits=3000 # 每个position用这么多计算量
%cd ~/work/data
!./katago benchmark -model {model}.bin.gz -v {visits}